In [1]:
import AdventOfCode

In [2]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as A

input <- dayLines 12

In [3]:
import Data.Functor (($>))

data Move = N | S | E | W | L | R | F deriving (Eq, Show)
data Facing = North | South | East | West deriving (Eq, Show)
type Instr = (Move, Int)

parseInstr = do
    m <- A.choice
        [ "N" $> N
        , "S" $> S
        , "E" $> E
        , "W" $> W
        , "L" $> L
        , "R" $> R
        , "F" $> F
        ]
    amt <- A.decimal
    pure (m, amt)

In [4]:
data Ship = Ship { facing :: Facing, coords :: (Int, Int) } deriving (Eq, Show)

steps = map (parsed parseInstr) input

move :: Ship -> Instr -> Ship
move (Ship f c@(x,y)) instr = case fst instr of
    L -> Ship (moveLeft f (snd instr)) c
    R -> Ship (moveRight f (snd instr)) c
    F -> Ship f (moveForward f c instr)
    _ -> Ship f (moveDirection c instr)
    where
        moveLeft :: Facing -> Int -> Facing
        moveLeft face 0 = face
        moveLeft face n = case face of
            North -> moveLeft West (n-90)
            West -> moveLeft South (n-90)
            South -> moveLeft East (n-90)
            East -> moveLeft North (n-90)

        moveRight face 0 = face
        moveRight face n = case face of
            North -> moveRight East (n-90)
            East -> moveRight South (n-90)
            South -> moveRight West (n-90)
            West -> moveRight North (n-90)

        moveForward face (x,y) (F,n) = case face of
            North -> (x,y+n)
            West -> (x-n,y)
            East -> (x+n,y)
            South -> (x,y-n)

        moveDirection (x,y) (d,n) = case d of
            N -> (x,y+n)
            W -> (x-n,y)
            E -> (x+n,y)
            S -> (x,y-n)

In [5]:
import Data.List (foldl')

final = foldl' move (Ship East (0,0)) steps

manhattan (x,y) = abs x + abs y

manhattan $ coords final

757

In [6]:
data Ship' = Ship' { waypoint :: (Int, Int), coords' :: (Int, Int) } deriving (Eq, Show)

rotateWaypointLeft (wx,wy) 0 = (wx,wy)
rotateWaypointLeft (wx,wy) n = rotateWaypointLeft (-1*wy,wx) (n-90)

rotateWaypointRight (wx,wy) 0 = (wx,wy)
rotateWaypointRight (wx,wy) n = rotateWaypointRight (wy,-1*wx) (n-90)

moveWaypoint (wx,wy) (d,n) = case d of
    N -> (wx,wy+n)
    S -> (wx,wy-n)
    E -> (wx+n,wy)
    W -> (wx-n,wy)

moveShip (wx,wy) (x,y) n = (x+(n*wx),y+(n*wy))

move' (Ship' w@(wx,wy) c@(x,y)) instr = case fst instr of
    L -> Ship' (rotateWaypointLeft w (snd instr)) c
    R -> Ship' (rotateWaypointRight w (snd instr)) c
    F -> Ship' w (moveShip w c (snd instr))
    _ -> Ship' (moveWaypoint w instr) c

In [7]:
final = foldl' move' (Ship' (10,1) (0,0)) steps

manhattan $ coords' final

51249